# 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import re
import glob

In [ ]:
not_buy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/5.미드프로젝트/간편식_HMR_을_구입하지_않는_이유_2021-2022.csv', encoding = 'utf-8')

not_buy.shape

(32, 23)

# 데이터 전처리

In [ ]:
not_buy.head(5)

,특성별(1),특성별(2),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,...,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10
0,특성별(1),특성별(2),가격이 비싸서,가족의 식사는 직접 조리 (요리) 해야 한다고 생각해서,맛이 없어서,원재료의 원산지와 품질이 의심스러워서/ 믿음이 가지 않아서,균형잡힌 영양섭취를 할 수 없을 것 같아서,어떤 제품이 있는지 어디서 파는지 몰라서,안전성(신선도 유통기한 등)이 염려되어서,첨가물이 걱정 되어서,...,가족의 식사는 직접 조리 (요리) 해야 한다고 생각해서,맛이 없어서,원재료의 원산지와 품질이 의심스러워서/ 믿음이 가지 않아서,균형잡힌 영양섭취를 할 수 없을 것 같아서,어떤 제품이 있는지 어디서 파는지 몰라서,안전성(신선도 유통기한 등)이 염려되어서,첨가물이 걱정 되어서,제품이 다양하지 않아서,없음,모름
1,전체,소계,8,37.5,7,18,4.4,4.3,13,7.7,...,17.5,14.2,14.1,11.3,5.5,9.3,5.5,3.1,0.4,0.5
2,거주지역별,수도권,9.4,43.7,7.7,16,0,5.5,12.8,4.8,...,18.6,16.6,10.7,12.2,4,8.5,6.2,3.7,0.5,1
3,거주지역별,충청권,0,0,0,24.5,29.9,0,21.1,24.5,...,19,14.5,14.2,8.6,11.7,7.1,1.3,3.2,0,0
4,거주지역별,호남권,0,0,0,0,0,0,0,0,...,15.5,12.3,16.1,17.4,7.1,15.5,5,0.9,0,0


In [ ]:
not_buy = not_buy.drop('특성별(1)', axis=1)
not_buy.drop([8,9,22,23,24,25,26,27,28], axis = 0, inplace = True)
not_buy.columns = not_buy.columns.map(lambda x: x.split('.')[0])
not_buy = not_buy.T.reset_index()

In [ ]:
col_list = ['year', '항목', '소계', '수도권', '충청권', '호남권', '대경권', '동남권', '강원권', '1인', '2인', '3인', '4인 이상', '남성', '여성', '20대', '30대', '40대', '50대', '60대 이상','200만원 미만', '200~400만원 미만','400~600만원 미만','600만원 이상']
not_buy.columns = col_list
not_buy.drop([0], axis = 0, inplace = True)

In [ ]:
not_buy.tail(2)

,year,항목,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,...,여성,20대,30대,40대,50대,60대 이상,200만원 미만,200~400만원 미만,400~600만원 미만,600만원 이상
20,2022,없음,0.4,0.5,0,0,0,1.1,0,0.4,...,0.3,0,0.4,0,0.8,0.3,0,0.2,0.3,0.8
21,2022,모름,0.5,1,0,0,0,0,0,1,...,0.8,0,0,0,0.7,1,0,0,0,1.6


In [ ]:
for col in not_buy.columns[2:]:                 # 2번쨰 열('소계')부터
    not_buy[col] = pd.to_numeric(not_buy[col])  # float 형식으로 변환

## 항목 일치작업

- 2021년도와 2022년도 항목 중 불일치 하는 부분 확인
    - 2021년도 : '기타' 항목 존재
    - 2022년도 : '없음', '모름' 항목 존재

- '없음'과 '모름'을 합하여 '기타'로 항목 일치시키고자 함

- 컬럼을 슬라이싱하여 작업
    - 'year'와 '항목' 컬럼은 숫자형이 아니기 때문

In [ ]:
# 'year','항목' 자르기
col_left = not_buy.loc[:,['year','항목']]
col_left.tail(2)

,year,항목
20,2022,없음
21,2022,모름


In [ ]:
# '소계' 부터 '600만원 이상'까지 자르기
col_right = not_buy.loc[:, '소계':'600만원 이상']
col_right.tail(2)

,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,2인,3인,...,여성,20대,30대,40대,50대,60대 이상,200만원 미만,200~400만원 미만,400~600만원 미만,600만원 이상
20,0.4,0.5,0.0,0.0,0.0,1.1,0.0,0.4,0.5,0.2,...,0.3,0.0,0.4,0.0,0.8,0.3,0.0,0.2,0.3,0.8
21,0.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,...,0.8,0.0,0.0,0.0,0.7,1.0,0.0,0.0,0.0,1.6


In [ ]:
# 2022년도의 '없음', '모름'을 합하여 '기타'로 처리
# 계산한 값을 20번째 행에 출력
col_right.loc[20, : ] = col_right.loc[20:22, :].sum(axis=0)
col_right.tail(2)

,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,2인,3인,...,여성,20대,30대,40대,50대,60대 이상,200만원 미만,200~400만원 미만,400~600만원 미만,600만원 이상
20,0.9,1.5,0.0,0.0,0.0,1.1,0.0,1.4,1.0,0.2,...,1.1,0.0,0.4,0.0,1.5,1.3,0.0,0.2,0.3,2.4
21,0.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,...,0.8,0.0,0.0,0.0,0.7,1.0,0.0,0.0,0.0,1.6


In [ ]:
# 슬라이싱했던 컬럼 합치기
not_buy2 = pd.concat( [col_left, col_right], axis=1 )

In [ ]:
# 필요없어진 21번째 행('모름') 삭제
not_buy2 = not_buy2.drop(21, axis=0)

In [ ]:
# '없음'값을 '기타'로 바꾸고 확인
not_buy2.loc[20, '항목'] = '기타'
not_buy2.tail(2)

,year,항목,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,...,여성,20대,30대,40대,50대,60대 이상,200만원 미만,200~400만원 미만,400~600만원 미만,600만원 이상
19,2022,제품이 다양하지 않아서,3.1,3.7,3.2,0.9,1.3,4.2,3.1,3.1,...,2.4,0.5,3.6,3.1,3.6,2.9,0.9,2.8,3.3,2.9
20,2022,기타,0.9,1.5,0.0,0.0,0.0,1.1,0.0,1.4,...,1.1,0.0,0.4,0.0,1.5,1.3,0.0,0.2,0.3,2.4


In [ ]:
not_buy2.drop("index", axis = 1, inplace = True)
not_buy2 = not_buy2.reset_index(drop=True)

# csv파일로 저장

In [ ]:
not_buy2.to_csv('/content/drive/MyDrive/Colab Notebooks/5.미드프로젝트/간편식_구입하지않는_이유.csv', index=False)